In [ ]:
import pandas as pd
from sklearn import preprocessing
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import GridSearchCV

In [ ]:
tweets = pd.read_csv("training_twitter_x_y_train.csv")
tweets

In [ ]:
documents = tweets[['text','airline_sentiment']]

In [ ]:
import numpy as np
documents = np.array(documents)

In [ ]:
documents = [(word_tokenize(word),sentiment) for word,sentiment in documents]

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [ ]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if(tag.startswith('J')):
        return wordnet.ADJ
    elif(tag.startswith('V')):
        return wordnet.VERB
    elif(tag.startswith('N')):
        return wordnet.NOUN
    elif(tag.startswith('R')):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
from nltk import pos_tag
w = 'better'
pos_tag([w])

In [ ]:
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)
stops

In [ ]:
def clean_review(words):
    output_words = []
    for w in words:
        if(w.lower() not in stops):
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w,pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word)
    return output_words  

In [ ]:
documents = [(clean_review(document),sentiment) for document,sentiment in documents]

In [ ]:
training_doc = documents

In [ ]:
tweets_test = pd.read_csv("test_twitter_x_test.csv")
documents1 = tweets_test['text']

In [ ]:
all_words = []
for doc in training_doc:
    all_words += doc[0]

In [ ]:
import nltk

In [ ]:
freq = nltk.FreqDist(all_words)
common  = freq.most_common(3000)
features = [i[0] for i in  common]

In [ ]:
def get_feature_dict(words):
    word_set = set(words)
    current_features = {}
    for w in features:
        current_features[w] = w in word_set    
    return current_features   

In [ ]:
training_data = [(get_feature_dict(doc),category) for doc,category in training_doc]

In [ ]:
from nltk import NaiveBayesClassifier

In [ ]:
clf = NaiveBayesClassifier.train(training_data)

In [ ]:
y_pred = []
for tweet in documents1:
      y_pred.append(clf.classify(get_feature_dict(clean_review(tweet.split()))))

In [ ]:
y_pred

In [ ]:
import numpy as np
np.savetxt("pred_twitter_sentiment1.csv",y_pred,fmt = "%s",delimiter=",")